In [10]:
import tensorflow as tf
# for load your model
from tensorflow import keras

#for load tokenizer
import pickle

# For reading uploaded file
from google.colab import files, drive
import io

# for convert news sample in numpy format
import numpy as np

In [5]:
# This library is needed for load indo bert tokenizer
!pip install transformers

     |████████████████████████████████| 4.2 MB 25.9 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 6.6 MB 65.7 MB/s 
     |████████████████████████████████| 84 kB 4.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# Access your model and tokenizer from drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/true-sight/model/indobert-base-p1-87')

In [6]:
with open('/content/drive/MyDrive/true-sight/model/indobert-base-p1-tokenizer-87.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

In [7]:
# This is utility function to encode news sample for prediction
def bert_encode(data, max_len) :
    input_ids = [] 
    attention_masks = []
    
    for i in range(len(data)):
        encoded = loaded_tokenizer.encode_plus(data[i],
                                        add_special_tokens=True,
                                        max_length=max_len,
                                        pad_to_max_length=True,
                                        return_attention_mask=True)
        
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

In [8]:
# Feel free to change this news sample
news_sample = ["Ikan lele merupakan salah satu makanan favorit di Indonesia. Selain harganya murah, rasanya juga sangat enak. Meski demikian, ada sebagian masyarakat yang takut menikmati masakan dari ikan air tawar tersebut. Mereka beranggapan jika ikan lele penyebab kanker dan penyakit lainnya.",
               "Jakarta, Penyakit stroke saat ini bukanlah penyakit yang langka mengingat bagaimana gaya hidup masyarakat saat ini. Stres, jarang berolah raga, minum minuman beralkohol, makan tidak teratur, dan memilih makanan cepat saji sudah menjadi hal biasa. Dan ternyata hal-hal inilah yang memicu penyakit stroke. Anda pun wajib waspada terhadap gejalanya. Pingsan tiba-tiba atau mengalami kemiringan mulut merupakan salah satu tanda-tanda stroke. Orang-orang seperti ini harus cepat mendapatkan pertolongan pertama.",
               "Pokemon aku yahudi adalah salah satu hal yang sering di share di grup sebutan favorit di indonesia",
               "Pokemon aku yahudi: CNN Indonesia",
               "Seorang wanita yang tengah hamil delapan bulan, Fitri (32), berhasil selamat dari peristiwa tanah longsor di Kampung Pasir Pogor, Desa Cipelang, Cijeruk, Kabupaten Bogor, Jawa Barat. Fitri sempat tertimbun longsor selama lima menit sebelum ditemukan warga.Ada lima menit saya sama anak-anak terjebak, sebelum ditolong warga. Pada ngeliat kepala anak-anak terus diangkat. Suami saya nggak tertolong, posisinya dekat pintu dapur, ungkap Fitri saat ditemui di tempatnya mengungsi tak jauh dari lokasi kejadian, Ahad.",
               "Innalilahi wainnailaihi Rojiun … Tragis Lettu Marinir Muhammad Ikbal Gugur Ditembak KKB Saat Sujud Sholat Isya",
               "Maudy Ayunda mengejutkan publik usai dikabarkan resmi menikah dengan kekasihnya pada Minggu (22/5). Hal itu muncul setelah dirinya selama ini jarang mengungkapkan hubungan asmara dengan seseorang.",
               "Covid merupakan virus buatan ukraina: CNN Indonesia",
               "Presiden Jokowi menghadang tank sendirian di ukraina",
               "Subhanallah pakdhe menghadang tank sendirian di kerusuhan ukraina"]

In [12]:
test_input = bert_encode(news_sample, 60)
preds = loaded_model.predict(test_input)
preds

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


array([[4.7885832e-01],
       [8.3993584e-02],
       [9.6535122e-01],
       [9.9995488e-01],
       [1.1664033e-03],
       [9.9996209e-01],
       [2.8884411e-04],
       [9.9937975e-01],
       [9.9847442e-01],
       [6.4038438e-01]], dtype=float32)

In [14]:
THRESHOLD = 0.70 
pred_df = pd.DataFrame(columns=['news', 'prediction'])
pred_df['news'] = news_sample
pred_df['prediction'] = preds

# If the probability is more than treshold it will labeled as hoax else fact
pred_df['prediction'] = pred_df['prediction'].apply(lambda x: 'Hoax' if x > THRESHOLD else 'Fakta')

pred_df

,news,prediction
0,Ikan lele merupakan salah satu makanan favorit...,Fakta
1,"Jakarta, Penyakit stroke saat ini bukanlah pen...",Fakta
2,Pokemon aku yahudi adalah salah satu hal yang ...,Hoax
3,Pokemon aku yahudi: CNN Indonesia,Hoax
4,Seorang wanita yang tengah hamil delapan bulan...,Fakta
5,Innalilahi wainnailaihi Rojiun … Tragis Lettu ...,Hoax
6,Maudy Ayunda mengejutkan publik usai dikabarka...,Fakta
7,Covid merupakan virus buatan ukraina: CNN Indo...,Hoax
8,Presiden Jokowi menghadang tank sendirian di u...,Hoax
9,Subhanallah pakdhe menghadang tank sendirian d...,Fakta
